In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.optimizers import Adam
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    age=int(ds.PatientAge)
    position=ds.PatientPosition
    body_part=ds.BodyPartExamined
    modality=ds.Modality
    if (age>20) and (age<70) :
        print("age meets specifications")
    else:
        print("age doesn't meet specifications")
        #assert False
        
    if (body_part=='CHEST'):
        print("body part meets specification")
    else:
        print("body part doesn't meet specification")
        assert False
    if (position in ['AP','PA']) :
        print("patient position meets specifications")
    else:
        print("patient position doesn't meet specifications")
        #assert False
    if (modality =="DX") :
        print("modality meets specifications")
    else:
        print("modality doesn't meet specifications")
        assert False
    print("some assertion errors are commented for simplicity of test")
    return ds.pixel_array
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size):
    img=img.reshape([img.shape[0],img.shape[1],1])/255.
    img_tensor=tf.convert_to_tensor(img)
    rgb_im=tf.image.grayscale_to_rgb(img_tensor).numpy()
    im2=tf.image.resize(rgb_im, [img_size[1],img_size[2]]).numpy()
    proc_img= im2.reshape(img_size)
    #print(im2.shape)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model  = keras.models.model_from_json(loaded_model_json)
    loaded_model.load_weights(weight_path)
    return loaded_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prob_Y = my_model.predict(img, verbose = True)   
    prediction=int(prob_Y.flatten()>thresh)
    return prediction 

In [3]:
#im = check_dicom('test1.dcm') 
#plt.imshow(im)
model_path ="my_model_517.json" #path to saved model
weight_path ='delivery_weights0517.h5' #path to saved best weights
my_model = load_model(model_path, weight_path)#loads model


In [47]:
# d=pydicom.dcmread('test6.dcm')
# d

In [4]:
#IMG_SIZE=[1,224,224,3]
#img = check_dicom("test3.dcm")
#print(img.shape[0])
# img=img.reshape((img.shape[0],img.shape[1],1))
# print(img.shape)
# ten_im=tf.convert_to_tensor(img)
# rgb_im=tf.image.grayscale_to_rgb(ten_im)
# img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
# img_proc.shape

In [6]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

IMG_SIZE=[1,224,224,3] # This might be different if you did not use vgg16
img_mean = 0.5# loads the mean image value they used during training preprocessing
img_std = 0.5# loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)#loads model
thresh =0.935 #loads the threshold they chose for model classification 

#use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    if img is None:
        continue
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
age doesn't meet specifications
body part meets specification
patient position meets specifications
modality meets specifications
some assertion errors are commented for simplicity of test
1/1 [==============================] - 1s 611ms/step
0
Load file test2.dcm ...
age meets specifications
body part meets specification
patient position meets specifications
modality meets specifications
some assertion errors are commented for simplicity of test
1/1 [==============================] - 1s 541ms/step
0
Load file test3.dcm ...
age doesn't meet specifications
body part meets specification
patient position meets specifications
modality meets specifications
some assertion errors are commented for simplicity of test
1/1 [==============================] - 1s 520ms/step
1
Load file test4.dcm ...
age doesn't meet specifications
body part doesn't meet specification


AssertionError: 